In [3]:
import mlflow
import torch

name = "track-gradients"
mlflow.set_tracking_uri("/Users/tanguanyu/UCC-DRN-Pytorch/mnist/mlruns")
experiment_id = "511211072178994368"
runs = mlflow.search_runs(experiment_names=["track-gradients"], output_format="list")

In [7]:
run = runs[0].to_dictionary()

In [3]:

import torch
import sys
sys.path.append("../mnist")
model = torch.load("/Users/tanguanyu/UCC-DRN-Pytorch/mnist/mlruns/511211072178994368/a7bb8db4ddb84ca7b1a7cc593971ed02/artifacts/best_model.pth/data/model.pth")

/var/folders/s_/yqk13gjs6b786w1dzl2sv9580000gn/T/ipykernel_935/2456577292.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("/Users/tanguanyu/UCC-DRN-Py

In [4]:
model

DRNOnlyModel(
  (drn): Sequential(
    (0): DRN(in_features=10, in_bins=11, out_features=10, out_bins=100)
    (1): DRN(in_features=10, in_bins=100, out_features=1, out_bins=4)
    (2): Flatten(start_dim=1, end_dim=-1)
  )
)

In [6]:
print(model.drn)

Sequential(
  (0): DRN(in_features=10, in_bins=11, out_features=10, out_bins=100)
  (1): DRN(in_features=10, in_bins=100, out_features=1, out_bins=4)
  (2): Flatten(start_dim=1, end_dim=-1)
)


In [8]:
import os
os.getcwd()

'/Users/tanguanyu/UCC-DRN-Pytorch/notebooks'

/var/folders/s_/yqk13gjs6b786w1dzl2sv9580000gn/T/ipykernel_935/529976608.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load("../mnist/drn_only.pth

In [14]:
from mnist.model import DRNOnlyModel
from hydra import compose, initialize

In [16]:
initial_params = {
        "hidden_q": 10,
        "num_bins": 11,
        "lr": 0.0498,
        "num_layers": 1,
        "num_nodes": 10
    }

defaults = {
                    "num_bins": {
                        "type": "int",
                        "range": [5,100],
                        "aliases": [
                            "model.drn.num_bins",
                            "args.num_bins",
                            "model.kde_model.num_bins"
                        ]
                    },
                    "lr": {
                        "type": "float",
                        "range": [0.0001, 0.1],
                        "aliases": ["args.learning_rate"]
                    },
                    "hidden_q": {
                        "type": "int",
                        "range": [4, 100],
                        "aliases": ["model.drn.hidden_q"]
                    },
                    "num_layers": {
                        "type": "int",
                        "range": [1, 10],
                        "aliases": ["model.drn.num_layers"]
                    },
                    "num_nodes": {
                        "type": "int",
                        "range": [1, 10],
                        "aliases": ["model.drn.num_nodes"]
                    }
                }
                
with initialize(version_base=None, config_path="../configs"):
    cfg = compose(config_name="train_drn")
for key, value in defaults.items():
    v = initial_params[key]
    for a in value["aliases"]:
        exec(f"cfg.{a} = {v}")
model = DRNOnlyModel(cfg)

In [17]:
model.drn

Sequential(
  (0): DRN(in_features=10, in_bins=11, out_features=10, out_bins=10)
  (1): DRN(in_features=10, in_bins=10, out_features=1, out_bins=4)
  (2): Flatten(start_dim=1, end_dim=-1)
)

In [32]:
model

DRNOnlyModel(
  (drn): Sequential(
    (0): DRN(in_features=10, in_bins=11, out_features=10, out_bins=10)
    (1): DRN(in_features=10, in_bins=10, out_features=1, out_bins=4)
    (2): Flatten(start_dim=1, end_dim=-1)
  )
)

In [3]:
import os

os.chdir("../mnist")

In [4]:
os.getcwd()

'/Users/tanguanyu/UCC-DRN-Pytorch/mnist'

In [12]:
from collections import defaultdict

import mlflow
import torch
import numpy as np

experiment_name = "ucc_drn_goated_init"
mlflow.set_tracking_uri("/Users/tanguanyu/UCC-DRN-Pytorch/mnist/mlruns")
runs = mlflow.search_runs(experiment_names=[experiment_name], output_format="list")
experiment = mlflow.set_experiment(experiment_name=experiment_name)
experiment_id = experiment.experiment_id
run_ids = [
    run.info.run_id for run in runs
]
for id in run_ids:
    prefix = f"mlruns/{experiment_id}/{id}"
    try:
        with open(f"{prefix}/metrics/eval_ucc_acc") as file:
            eval_ucc = file.readlines()

        init_model = torch.load(f"{prefix}/artifacts/init_model/data/model.pth", weights_only=False)
        trained_model = torch.load(f"{prefix}/artifacts/best_model/data/model.pth", weights_only=False)
        drn_model = init_model.ucc_classifier
        print(id)
        print("".join(eval_ucc))
        param_dict = defaultdict(list)
        n_list = ["W", "bq", "ba", "lama", "lamq"]
        for name, param in drn_model.named_parameters():
            for n in n_list:
                if n in name:
                    param_dict[f"{n}_max"].append(param.max().detach().cpu().numpy())
                    param_dict[f"{n}_min"].append(param.min().detach().cpu().numpy())
        for p, value in param_dict.items():
            print(f"{p}: {np.mean(value)}")
    except:
        continue

In [ ]:
run_id

[]